In [1]:
import pandas as pd
import numpy as np
import time
import copy
import datetime
from sklearn import linear_model
from sklearn.cross_validation import cross_val_predict
from sklearn.naive_bayes import GaussianNB

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
course_id = 'KNU/101/2014_T2'
user_id = 50444
sm = pd.read_csv('F:\Projects\PA\Scripts\data\courseware_studentmodule.csv')

In [3]:
users = pd.read_csv('F:\Projects\PA\Scripts\data\users.csv')

In [4]:
cert = pd.read_csv('F:\Projects\PA\Scripts\data\certificates_generatedcertificate.csv')

C:\ProgramData\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
sm

,id,module_type,module_id,student_id,state,grade,created,modified,max_grade,done,course_id
0,1,problem,i4x://edX/DemoX/problem/303034da25524878a2e66f...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
1,2,problem,i4x://edX/DemoX/problem/932e6f2ce8274072a355a9...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
2,3,problem,i4x://edX/DemoX/problem/9cee77a606ea4c1aa5440e...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
3,4,problem,i4x://edX/DemoX/problem/0d759dee4f9d459c895613...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
4,5,problem,i4x://edX/DemoX/problem/75f9562c77bc4858b61f90...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
5,6,problem,i4x://edX/DemoX/problem/Sample_ChemFormula_Pro...,8,"{""correct_map"": {}, ""seed"": 1, ""done"": false, ...",\b,2014-08-05 13:39:19,2014-11-16 15:24:34,\b,na,edX/DemoX/Demo_Course
6,7,problem,i4x://edX/DemoX/problem/Sample_Algebraic_Problem,8,"{""correct_map"": {""i4x-edX-DemoX-problem-Sample...",1,2014-08-05 13:39:19,2014-11-16 15:27:39,1,na,edX/DemoX/Demo_Course
7,8,problem,i4x://edX/DemoX/problem/a0effb954cca4759994f1a...,8,"{""seed"": 1, ""input_state"": {""i4x-edX-DemoX-pro...",\b,2014-08-05 13:39:19,2014-08-05 13:39:19,\b,na,edX/DemoX/Demo_Course
8,9,problem,i4x://edX/DemoX/problem/d2e35c1d294b4ba0b3b104...,8,"{""correct_map"": {""i4x-edX-DemoX-problem-d2e35c...",1,2014-08-05 13:39:19,2014-09-17 10:38:28,1,na,edX/DemoX/Demo_Course
9,10,problem,i4x://edX/DemoX/problem/c554538a57664fac80783b...,8,"{""correct_map"": {""i4x-edX-DemoX-problem-c55453...",1,2014-08-05 13:39:19,2014-09-17 10:37:29,1,na,edX/DemoX/Demo_Course


In [6]:
sm = sm[sm.module_type == 'problem']
sm = sm.rename(columns = {'student_id':'user_id'})
sm.grade = map(lambda x: x if x != '\\b' else 0,sm.grade.tolist())
cert = cert[cert.course_id == course_id]
registered = cert.user_id.unique()
users = users[users.id.isin(registered)]

In [7]:
#fit
##__getProblemResultFeatures
start_time = time.time()
modules = sm.module_id.unique().tolist()
result = []
for module in modules:
    first = sm[sm.module_id == module].created.min()
    result.append({'name':module,'date':first})
labels = map(lambda x: x['name'],sorted(result,key=lambda x: x['date']))
rebased = sm.pivot(index='user_id',columns='module_id',values='grade')
rebased = rebased[labels].fillna(0)
res_features =  rebased
print '__getProblemResultFeatures finished in ' + str(time.time() - start_time)
##

__getProblemResultFeatures finished in 502.836999893


In [8]:
##__getUserDataFeatures
start_time = time.time()
u = pd.DataFrame(users[['gender','year_of_birth','level_of_education']],index=users.user_id)
c = cert.set_index(cert.user_id)[['grade','status']]
c['status'] = c.apply(lambda x: 1 if x.status == 'downloadable' else 0,1)
u_features = c.join(u)
print '__getUserDataFeatures finished in ' + str(time.time() - start_time)
##

__getUserDataFeatures finished in 1.80500006676


In [9]:
map(lambda x: x if x != '\\b' else 0,sm.grade.tolist())
#sm.grade.unique()

[0,
 0,
 0,
 0,
 0,
 0,
 '1',
 0,
 '1',
 '1',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '0',
 '6',
 '2',
 0,
 '6',
 '3',
 '3',
 '6',
 '3',
 '6',
 0,
 0,
 '1',
 0,
 '3',
 '6',
 '3',
 0,
 '2',
 '6',
 '6',
 '3',
 0,
 0,
 0,
 0,
 0,
 0,
 '6',
 '3',
 '3',
 '6',
 '3',
 '6',
 0,
 0,
 '6',
 '3',
 '3',
 '6',
 '0',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '3',
 '6',
 '3',
 '6',
 '3',
 '6',
 '6',
 '3',
 '3',
 '6',
 0,
 0,
 '2',
 0,
 '2',
 '6',
 '2',
 '6',
 '3',
 '6',
 '6',
 '3',
 0,
 0,
 0,
 0,
 '3',
 '6',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '6',
 '3',
 0,
 0,
 '2',
 0,
 '3',
 '4',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '6',
 '2',
 '2',
 '6',
 '0',
 0,
 0,
 '3',
 0,
 0,
 '2',
 '1',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '6',
 '3',
 '0',
 '6',
 '3',
 '6',
 '3',
 '6',
 0,
 '0',
 0,
 0,
 '1',
 0,
 '2',
 0,
 '2',
 '1',
 0,
 0,
 0,
 0,
 '4',
 '3',
 '4',
 '3',
 '4',
 0,
 '3',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '3',
 '4',
 0,
 '4',
 '3',
 '1',
 0,
 0,
 0,
 0,
 0

In [10]:
##__lastPassedTest
start_time = time.time()
features = res_features.ix[user_id].tolist()
index = len(features)
for i in reversed(features):
    if i != 0:
        break
    index -= 1
if index == 0:
    index = len(res_features.columns)
last = index
print '__lastPassedTest finished in ' + str(time.time() - start_time)
##

__lastPassedTest finished in 0.0119998455048


In [65]:
start_time = time.time()
cols = res_features.columns[:last].tolist()
dataset = u_features.join(res_features)
data = dataset[cols].fillna(0)
Y = dataset.status
print '_final preparations finished in ' + str(time.time() - start_time)

_final preparations finished in 10.8890001774


In [12]:
model = GaussianNB()
start_time = time.time()
predicted = cross_val_predict(model, data, Y)
print 'cross_val_predict finished in ' + str(time.time() - start_time)

score = float(np.sum(predicted == Y))/len(data)
start_time = time.time()
model.fit(data,Y)
print 'fit finished in ' + str(time.time() - start_time)

cross_val_predict finished in 9.54699993134
fit finished in 2.90700006485


In [13]:
#predict

cols = res_features.columns.tolist()[:last]
start_time = time.time()
answer = model.predict(res_features[cols].ix[user_id].tolist())
print 'predict finished in ' + str(time.time() - start_time)
print score

predict finished in 1.42000007629
0.607830299856


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [14]:
dataset

,grade,status,gender,year_of_birth,level_of_education,i4x://edX/DemoX/problem/303034da25524878a2e66fb57c91cf85,i4x://edX/DemoX/problem/932e6f2ce8274072a355a94560216d1a,i4x://edX/DemoX/problem/9cee77a606ea4c1aa5440e0ea5d0f618,i4x://edX/DemoX/problem/0d759dee4f9d459c8956136dbde55f02,i4x://edX/DemoX/problem/75f9562c77bc4858b61f907bb810d974,...,i4x://OsvitaOnline/Eng101/problem/0fc00d2172c347e683369a3813f31ec1,i4x://OsvitaOnline/Eng101/problem/4ce46b5c9da048f2aba3b8336b58b5f7,i4x://Prometheus/103/problem/c79fcbbfb444436986dbc8f37d995180,i4x://Prometheus/103/problem/32c557b08d2b44a0a6b53b553eff48c4,i4x://Prometheus/103/problem/aa9afa9d37f84bd182353ef0c1282bae,i4x://Prometheus/103/problem/5dcde511f59d44c5a5e9447318640b58,i4x://Prometheus/103/problem/471d220653a24bde946cb07125652ac1,i4x://Prometheus/103/problem/fa539c440de3459c8ad4f39910ae54bf,i4x://Prometheus/103/problem/5b5860bcfa3b4f7dad8a7a59f2d0987e,i4x://Prometheus/103/problem/e09fe3fb88334075b272a202f08694f8
user_id,,,,,,,,,,,,,,,,,,,,,
47030,0.92,1,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0.89,1,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
113,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,0.15,0,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
122,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0.00,0,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
160,0.18,0,NaN,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
input = dataset.fillna(0)
output = dataset.status

#prediction test
model = GaussianNB()
start_time = time.time()
predicted = cross_val_predict(model, data, Y)
print 'cross_val_predict finished in ' + str(time.time() - start_time)

score = float(np.sum(predicted == Y))/len(data)
start_time = time.time()
model.fit(data,Y)
print 'fit finished in ' + str(time.time() - start_time)

cols = res_features.columns.tolist()[:last]
start_time = time.time()
answer = model.predict(res_features[cols].ix[user_id].tolist())
print 'predict finished in ' + str(time.time() - start_time)

cross_val_predict finished in 10.5170001984
fit finished in 2.59200000763
predict finished in 2.75499987602


C:\ProgramData\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


In [69]:
ex_cols = list(cols)
ex_cols.append('status')
print ex_cols[-1]
print ex_cols[-2]

status
i4x://Prometheus/103/problem/e09fe3fb88334075b272a202f08694f8


In [70]:
dataset_x = dataset.fillna(0)[ex_cols]

baseFrame = pd.DataFrame(dataset_x.values.astype(float), columns=dataset_x.columns, index=map(lambda x: str(x) + '_x0' ,dataset_x.index.tolist()))
for i in range(1,5):
    currentFrame = pd.DataFrame(dataset_x.values.astype(float), columns=dataset_x.columns, index=map(lambda x: str(x) + '_x' + str(i) ,dataset_x.index.tolist()))
    baseFrame = pd.concat([baseFrame, currentFrame])

In [74]:
print len(cols)
print len(baseFrame.columns)
print len(baseFrame)

1076
1077
76370


In [75]:
input = baseFrame[cols]
output = baseFrame.status

#prediction test
model = GaussianNB()
start_time = time.time()
predicted = cross_val_predict(model, input, output)
print 'cross_val_predict finished in ' + str(time.time() - start_time)

score = float(np.sum(predicted == output))/len(input)
start_time = time.time()
model.fit(input,output)
print 'fit finished in ' + str(time.time() - start_time)

cols = res_features.columns.tolist()[:last]
start_time = time.time()
answer = model.predict(res_features[cols].ix[user_id].tolist())
print 'predict finished in ' + str(time.time() - start_time)

MemoryError: 

In [41]:
a = pd.DataFrame(dataset.values, columns=dataset.columns, index=map(lambda x: str(x) + '_x0' ,dataset.index.tolist())).fillna(0)[cols].values

In [43]:
uo = np.unique(a.ravel())
uo

array([0L, '0', '0.1', '0.103448275862069', '0.2', '0.3',
       '0.310344827586207', '0.4', '0.448275862068966', '0.5', '0.55',
       '0.6', '0.620689655172414', '0.655172413793103',
       '0.666666666666667', '0.724137931034483', '0.733333333333333',
       '0.75', '0.76', '0.793103448275862', '0.8', '0.827586206896552',
       '0.85', '0.862068965517241', '0.888888888888889', '0.9', '0.92',
       '0.925925925925926', '0.931034482758621', '0.933333333333333',
       '0.95', '0.96551724137931', '0.966666666666667', '1', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '4', '5',
       '6', '7', '8', '9'], dtype=object)

In [46]:
a.astype(float)

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [ ]:
cross_validation_timing = []
fit_timing = []
predict_timing = []

start_time = time.time()
for i in range(0,100):
    baseFrame = pd.DataFrame(dataset.values, columns=dataset.columns, index=map(lambda x: str(x) + '_x' ,dataset.index.tolist()))
    for i in range(0,i):
        currentFrame = pd.DataFrame(dataset.values, columns=dataset.columns, index=map(lambda x: str(x) + '_x' + str(i) ,dataset.index.tolist()))
        baseFrame = pd.concat([baseFrame, currentFrame])
        
    input = baseFrame.fillna(0)
    output = baseFrame.status

    #prediction test
    model = GaussianNB()
    start_time = time.time()
    predicted = cross_val_predict(model, data, Y)
    print 'cross_val_predict finished in ' + str(time.time() - start_time)

    score = float(np.sum(predicted == Y))/len(data)
    start_time = time.time()
    model.fit(data,Y)
    print 'fit finished in ' + str(time.time() - start_time)

    cols = res_features.columns.tolist()[:last]
    start_time = time.time()
    answer = model.predict(res_features[cols].ix[user_id].tolist())
    print 'predict finished in ' + str(time.time() - start_time)
    
    print str(i) + 'is done after ' + str(time.time() - start_time) + ' seconds after start'

In [66]:
for i in range(1,2):
    print 'hello'

hello
